In [11]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from shapely.wkt import loads
from city_road_network.utils.utils import get_data_subdir
from city_road_network.algo.gravity_model import run_gravity_model, get_attr_error, get_prod_error
from city_road_network.config import default_crs

In [12]:
city_name = "moscow"
data_dir = get_data_subdir(city_name)

In [13]:
df_zones = pd.read_csv(os.path.join(data_dir, "zones_upd.csv"), index_col=0)
df_zones["geometry"] = df_zones["geometry"].apply(loads)
df_zones["centroid"] = df_zones["centroid"].apply(loads)

print(len(df_zones.loc[df_zones["production"] == 0]))
print(len(df_zones.loc[df_zones["poi_attraction"] == 0]))

df_zones.loc[df_zones["production"] == 0, "production"] = 1
df_zones.loc[df_zones["poi_attraction"] == 0, "poi_attraction"] = 1

1
1


In [14]:
zones_gdf = gpd.GeoDataFrame(df_zones, crs=default_crs)
zones_gdf

,name,geometry,poi_count,poi_attraction,pop,households,vehicles,production,centroid
0,район Кунцево,"POLYGON ((36.91270 55.74751, 36.90939 55.74836...",0,1.000000,0.000000,0.000000,0.000000,1.000000,POINT (37.011244754896026 55.73195412741748)
1,район Крылатское,"POLYGON ((37.36539 55.77181, 37.37506 55.76732...",244,4199.958303,85674.682041,35697.784184,28558.227347,6425.601153,POINT (37.42366416645674 55.76322141006385)
2,Мещанский район,"POLYGON ((37.62091 55.75899, 37.62159 55.75910...",856,14288.999330,63717.706840,26549.044517,21239.235613,4778.828013,POINT (37.628778583004916 55.780012408073404)
3,район Сокольники,"POLYGON ((37.69532 55.81171, 37.69510 55.81173...",398,7739.715655,72162.528133,30067.720055,24054.176044,5412.189610,POINT (37.677578248842835 55.79987699140851)
4,район Хорошёво-Мнёвники,"POLYGON ((37.50251 55.79060, 37.50191 55.79019...",384,8145.095417,181964.255450,75818.439771,60654.751817,13647.319159,POINT (37.464191650603 55.772902124215626)
...,...,...,...,...,...,...,...,...,...
113,район Метрогородок,"POLYGON ((37.67021 55.83874, 37.67126 55.83846...",95,1796.400408,168597.744825,70249.060344,56199.248275,12644.830862,POINT (37.73849677001702 55.8428508731295)
114,район Гольяново,"POLYGON ((37.75085 55.80217, 37.75223 55.80242...",423,7960.803384,168150.365708,70062.652378,56050.121903,12611.277428,POINT (37.7961997185952 55.82428756797183)
115,район Измайлово,"POLYGON ((37.74535 55.77005, 37.74804 55.77005...",476,9711.893288,66939.281775,27891.367406,22313.093925,5020.446133,POINT (37.77275147910152 55.783786725930554)
116,район Ивановское,"POLYGON ((37.83848 55.78838, 37.84088 55.78879...",253,5428.326173,137163.695820,57151.539925,45721.231940,10287.277186,POINT (37.8240270935332 55.771337213436695)


In [15]:
zones_gdf[zones_gdf["poi_attraction"] == 1]

,name,geometry,poi_count,poi_attraction,pop,households,vehicles,production,centroid
0,район Кунцево,"POLYGON ((36.91270 55.74751, 36.90939 55.74836...",0,1.0,0.0,0.0,0.0,1.0,POINT (37.011244754896026 55.73195412741748)


In [16]:
prod_array = np.array(zones_gdf["production"])
attr_array = np.array(zones_gdf["poi_attraction"])

In [17]:
prod_array.sum()

879826.6605792224

In [18]:
trip_mat = run_gravity_model(zones_gdf)

In [19]:
prod_error = get_prod_error(trip_mat, prod_array)
attr_error = get_attr_error(trip_mat, attr_array)
print(prod_error, attr_error)

8.66289030659238 9.30406022280108


In [20]:
np.save(os.path.join(data_dir, "trip_mat"), trip_mat)